# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [ ]:
!pip install ratsnlp

## 말뭉치 다운로드 및 전처리

오픈소스 파이썬 패키지 코포라(Korpora)를 활용해 BPE 수행 대상 말뭉치를 내려받고 전처리합니다. 실습용 말뭉치는 박은정 님이 공개하신 Naver Sentiment Movie Corpus(NSMC)입니다.

다음을 수행해 데이터를 내려받아 `nsmc`라는 변수로 읽어들입니다.

In [ ]:
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download=True)

다음을 수행하면 NSMC에 포함된 영화 리뷰(순수 텍스트)들을 지정된 경로에 저장해 둡니다.

In [ ]:
import os
def write_lines(path, lines):
    with open(path, 'w', encoding='utf-8') as f:
        for line in lines:
            f.write(f'{line}\n')

write_lines("/root/train.txt", nsmc.train.get_all_texts())
write_lines("/root/test.txt", nsmc.test.get_all_texts())

`train.txt`의 앞부분은 다음과 같이 생겼습니다.

In [ ]:
!head /root/train.txt

`test.txt`의 앞부분은 다음과 같이 생겼습니다.

In [ ]:
!head /root/test.txt

## GPT 토크나이저 구축

GPT 계열 모델이 사용하는 토크나이저는 Byte-level Byte Pair Encoding(BBPE)입니다. 다음을 수행하면 `nsmc` 데이터를 가지고 BBPE 어휘집합을 구축합니다. BBPE 어휘집합 구축에 시간이 걸리니 잠시 기다려주세요. 

In [ ]:
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files=["/root/train.txt", "/root/test.txt"],
    vocab_size=10000,
    special_tokens=["[PAD]"]
)
bytebpe_tokenizer.save_model("/root")

위의 코드 수행이 끝나면 `/root` 디렉토리에 `vocab.json`과 `merges.txt`가 생성됩니다. 전자는 바이트 레벨 BPE의 어휘 집합이며 후자는 바이그램 쌍의 병합 우선순위입니다. 

`vocab.json`은 다음과 같이 생겼습니다.

In [ ]:
!cat /root/vocab.json

`merges.txt`의 앞부분은 다음과 같이 생겼습니다.

In [ ]:
!head /root/merges.txt

## BERT 토크나이저 구축

BERT는 워드피스(wordpiece) 토크나이저를 사용합니다. 다음을 수행하면 BERT 모델이 사용하는 워드피스 어휘집합을 구축할 수 있습니다. 워드피스 어휘집합 구축에 시간이 걸리니 잠시만 기다려주세요.

In [ ]:
from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files=["/root/train.txt", "/root/train.txt"],
    vocab_size=10000,
)
wordpiece_tokenizer.save_model("/root")

위의 코드 수행이 끝나면 `/root` 디렉토리에 `vocab.txt`가 생성됩니다. `vocab.txt`의 앞부분은 다음과 같이 생겼습니다.

In [ ]:
!head /root/vocab.txt

## GPT 입력값 만들기

GPT 모델 입력값을 만들려면 Byte-level Byte Pair Encoding 어휘집합 구축 결과(`vocab.json`, `merges.txt`)가 필요합니다. 다음을 수행해 이미 만들어 놓은 BBPE 어휘집합을 포함한 GPT 토크나이저를 `tokenizer_gpt`라는 변수로 선언합니다.

In [ ]:
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained("/root")
tokenizer_gpt.pad_token = "[PAD]"

예시 문장 세 개를 각각 토큰화해보겠습니다.

In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

토큰화 결과를 확인합니다.

In [ ]:
tokenized_sentences

이번 배치의 크기가 3이라고 가정하고 이번 배치의 입력값을 만들어 보겠습니다.

In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_gpt(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

`batch_inputs`의 내용을 확인해보겠습니다.

In [ ]:
batch_inputs.keys()

In [ ]:
batch_inputs['input_ids']

In [ ]:
batch_inputs['attention_mask']

## BERT 입력값 만들기

BERT 모델 입력값을 만들려면 워드피스 어휘집합 구축 결과(`vocab.txt`)가 필요합니다. 다음을 수행해 위에서 이미 만들어 놓은 워드피스 어휘집합을 포함한 BERT 토크나이저를 `tokenizer_bert`라는 변수로 선언합니다.

In [ ]:
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(
    "/root", 
    do_lower_case=False,
)

예시 문장 세 개를 각각 토큰화해보겠습니다.

In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

토큰화 결과를 확인합니다.

In [ ]:
tokenized_sentences

이번 배치의 크기가 3이라고 가정하고 이번 배치의 입력값을 만들어 보겠습니다.

In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_bert(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

`batch_inputs`의 내용을 확인해보겠습니다.

In [ ]:
batch_inputs.keys()

In [ ]:
batch_inputs['input_ids']

In [ ]:
batch_inputs['attention_mask']

In [ ]:
batch_inputs['token_type_ids']